# Using Ollama

In [ ]:
import json
import os
import gradio as gr
import ollama

# =====================================================
# OLLAMA CONFIG
# =====================================================

MODEL = "gemma:2b"



# =====================================================
# FILE STORAGE SETUP
# =====================================================

FILE = "train_prices_india.json"

# Initialize file if not exists
if not os.path.exists(FILE):
    initial_prices = {
        "delhi-mumbai": 750,
        "delhi-kolkata": 680,
        "mumbai-bangalore": 820,
        "chennai-hyderabad": 450,
        "ahmedabad-jaipur": 520,
        "lucknow-patna": 300,
        "pune-goa": 350
    }
    with open(FILE, "w") as f:
        json.dump(initial_prices, f)

def load_prices():
    with open(FILE, "r") as f:
        return json.load(f)

def save_prices(data):
    with open(FILE, "w") as f:
        json.dump(data, f)

# =====================================================
# TOOL FUNCTIONS (DETERMINISTIC)
# =====================================================

def get_ticket_price(route):
    data = load_prices()
    route = route.lower()

    if route in data:
        return f" Train ticket price for {route.title()} is ₹{data[route]}"
    return f" No train ticket data available for {route.title()}."

def update_ticket_price(route, price):
    data = load_prices()
    data[route.lower()] = price
    save_prices(data)
    return f" Ticket price for {route.title()} updated to ₹{price}."

# =====================================================
# INTENT DETECTION
# =====================================================

def detect_intent(user_message):
    msg = user_message.lower().strip()

    if "price" in msg or "fare" in msg:
        return "get_price"

    if "update" in msg or "change" in msg:
        return "update_price"

    return "general"

# Extract route like "Delhi Mumbai"
def extract_route(user_message):
    words = user_message.lower().split()
    cities = [
        "delhi", "mumbai", "kolkata", "bangalore",
        "chennai", "hyderabad", "ahmedabad",
        "jaipur", "lucknow", "patna", "pune", "goa"
    ]

    found = [word for word in words if word in cities]

    if len(found) >= 2:
        return f"{found[0]}-{found[1]}"

    return None

def extract_price(user_message):
    words = user_message.split()
    for word in words:
        if word.isdigit():
            return float(word)
    return None

# =====================================================
# CHAT FUNCTION
# =====================================================

def chat(user_message, history):

    intent = detect_intent(user_message)

    # -------- GET PRICE --------
    if intent == "get_price":
        route = extract_route(user_message)

        if route is None:
            return "Please specify route like: Delhi Mumbai"

        return get_ticket_price(route)

    # -------- UPDATE PRICE --------
    if intent == "update_price":
        route = extract_route(user_message)
        price = extract_price(user_message)

        if route is None:
            return "Please specify route like: Delhi Mumbai"

        if price is None:
            return "Please provide a valid ticket price."

        return update_ticket_price(route, price)

    # -------- GENERAL QUESTION (LLM) --------
    messages = [
        {"role": "system", "content": "You are RailMitra AI, a helpful Indian railway assistant."},
        {"role": "user", "content": user_message}
    ]

    response = ollama.chat(
        model=MODEL,
        messages=messages
    )

    return response["message"]["content"]

# =====================================================
# GRADIO UI
# =====================================================

demo = gr.ChatInterface(
    fn=chat,
    title="🚆 RailMitra AI - Indian Train Ticket Assistant"
)

demo.launch()


# Using

In [ ]:
import gradio as gr
from groq import Groq
import json

# --- CONFIG ---
client = Groq(api_key="your api key")

MODEL = "llama-3.3-70b-versatile"   # Recommended

# --- TOOL FUNCTION ---
def get_train_fare(destination_city: str):
    print(f"\n Tool Triggered! Searching fare for: {destination_city}")

    prices = {
        "mumbai": "1,600",
        "jaipur": "650",
        "bengaluru": "2,400",
        "kolkata": "1,750",
        "varanasi": "980",
        "chennai": "2,100",
        "agra": "450",
        "amritsar": "800"
    }

    city = destination_city.lower().strip()
    price = prices.get(city)

    if price:
        return {"price": price, "currency": "INR", "status": "available"}
    else:
        return {"error": f"Fare not found for {city}", "status": "unknown"}


# --- TOOL SCHEMA (OpenAI Compatible Format) ---
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_train_fare",
            "description": "Get train fare from New Delhi to a destination city",
            "parameters": {
                "type": "object",
                "properties": {
                    "destination_city": {
                        "type": "string",
                        "description": "Name of the destination city"
                    }
                },
                "required": ["destination_city"]
            }
        }
    }
]


# --- CHAT FUNCTION ---
def chat(message, history):

    messages = [
        {
            "role": "system",
            "content": """
            You are RailSathi, a helpful Indian Railways assistant.
            When a user asks for ticket price or fare,
            you MUST call the get_train_fare tool.
            After getting the tool result,
            respond politely in natural language.
            Prices are always in INR.
            """
        }
    ]

    # Add history
    for msg in history:
        messages.append({
            "role": msg["role"],
            "content": msg["content"]
        })

    # Add new user message
    messages.append({"role": "user", "content": message})

    # First API call
    response = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        tools=tools,
        tool_choice="auto"
    )
    print(response.model_dump())
    with open("model_tools_reponse.json", "w") as f:
        json.dump(response.model_dump(mode="json"), f, indent=4)
    msg = response.choices[0].message

    # If tool is called
    if msg.tool_calls:
        tool_call = msg.tool_calls[0]
        arguments = json.loads(tool_call.function.arguments)

        # Execute tool
        result = get_train_fare(**arguments)

        # Add tool call + result to conversation
        messages.append(msg)
        messages.append({
            "role": "tool",
            "tool_call_id": tool_call.id,
            "content": json.dumps(result)
        })

        # Second API call (Final Answer)
        second_response = client.chat.completions.create(
            model=MODEL,
            messages=messages
        )

        return second_response.choices[0].message.content

    # If no tool needed
    return msg.content


# --- LAUNCH ---
if __name__ == "__main__":
    gr.ChatInterface(
        fn=chat,
        title="🚆 RailSathi AI (Groq Powered)",
        description="Ask for train fares from New Delhi (e.g., 'Price for Mumbai?').",
    ).launch()


# With Langchain

In [ ]:
import gradio as gr

from langchain_groq import ChatGroq
from langchain.tools import tool
from langchain.agents import initialize_agent, AgentType

# ----------------------------
# CONFIG
# ----------------------------

MODEL = "llama-3.3-70b-versatile"

llm = ChatGroq(
    model=MODEL,
    api_key="YOUR_GROQ_API_KEY",  
    temperature=0
)

# ----------------------------
# TOOL DEFINITION
# ----------------------------

@tool
def get_train_fare(destination_city: str) -> str:
    """
    Get train fare from New Delhi to a destination city.
    """

    print(f"\n🚆 Tool Triggered! Searching fare for: {destination_city}")

    prices = {
        "mumbai": "1,600",
        "jaipur": "650",
        "bengaluru": "2,400",
        "kolkata": "1,750",
        "varanasi": "980",
        "chennai": "2,100",
        "agra": "450",
        "amritsar": "800"
    }

    city = destination_city.lower().strip()
    price = prices.get(city)

    if price:
        return f"The train fare from New Delhi to {city.title()} is ₹{price} INR."
    else:
        return f"Sorry, fare not found for {city.title()}."

# ----------------------------
# AGENT SETUP
# ----------------------------

tools = [get_train_fare]

agent_executor = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.OPENAI_FUNCTIONS,  # Function calling agent
    verbose=True
)

# ----------------------------
# CHAT FUNCTION
# ----------------------------

def chat(message, history):
    response = agent_executor.invoke({"input": message})
    return response["output"]

# ----------------------------
# GRADIO UI
# ----------------------------

if __name__ == "__main__":
    gr.ChatInterface(
        fn=chat,
        title="🚆 RailSathi AI (LangChain + Groq)",
        description="Ask for train fares from New Delhi (e.g., 'Price for Mumbai?').",
    ).launch()
